In [75]:
import pandas as pd
import numpy as np
import random
p = 0.004  # 0.4% of the lines

import io
import requests
from datetime import datetime, timezone, timedelta
import datetime as dt
import time

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### __Passenger: Feature Engineering__

In [76]:
# import passengers dataset
pass_df = pd.read_csv('passengers.csv')
pass_df.info()
pass_df.head()

# keep the header, then take only 0.4% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
#passenger = pd.read_csv(
#         '../data/passengers.csv',
#         header=0, 
#         skiprows=lambda i: i>0 and random.random() > p
#)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350497 entries, 0 to 2350496
Data columns (total 38 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   departures_scheduled   float64
 1   departures_performed   float64
 2   payload                float64
 3   seats                  float64
 4   passengers             float64
 5   freight                float64
 6   mail                   float64
 7   distance               float64
 8   ramp_to_ramp           float64
 9   air_time               float64
 10  unique_carrier         object 
 11  airline_id             int64  
 12  unique_carrier_name    object 
 13  region                 object 
 14  carrier                object 
 15  carrier_name           object 
 16  carrier_group          int64  
 17  carrier_group_new      int64  
 18  origin_airport_id      int64  
 19  origin_city_market_id  int64  
 20  origin                 object 
 21  origin_city_name       object 
 22  origin_country    

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,2.0,4000.0,16.0,0.0,0.0,0.0,223.0,126.0,91.0,...,US,United States,6,651,1,2015,6,1,L,DU
1,0.0,1.0,2000.0,8.0,1.0,0.0,0.0,236.0,100.0,45.0,...,US,United States,6,651,1,2015,6,1,L,DU
2,0.0,1.0,3000.0,13.0,0.0,0.0,0.0,386.0,80.0,63.0,...,US,United States,6,641,1,2015,6,1,L,DU
3,0.0,1.0,3000.0,14.0,0.0,0.0,0.0,18.0,21.0,9.0,...,US,United States,6,641,1,2015,6,1,L,DU
4,0.0,1.0,2000.0,8.0,0.0,0.0,0.0,18.0,19.0,9.0,...,US,United States,6,651,1,2015,6,1,L,DU


In [77]:
# start time to measure the time of the program execution
start_time = time.time()

pass_df = pass_df.drop(columns=[
    'departures_scheduled', 'departures_performed', 'payload', 'seats', 
     'freight', 'mail', 'ramp_to_ramp', 'airline_id', 'unique_carrier_name', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new', 'origin_airport_id',
       'origin_city_market_id', 'origin',
       'origin_city_name', 'origin_country', 'origin_country_name',
       'dest_city_market_id', 'dest', 'dest_city_name',
       'dest_country', 'dest_country_name', 'aircraft_group', 'aircraft_type',
       'aircraft_config', 'year', 'distance_group', 'class',
       'data_source'
])

# rearrange columns
rearranged_columns = ['dest_airport_id', 'unique_carrier', 'month',
                      'passengers', 'distance', 'air_time']
pass_df = pass_df[rearranged_columns]

# check
pass_df.info()
pass_df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350497 entries, 0 to 2350496
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   dest_airport_id  int64  
 1   unique_carrier   object 
 2   month            int64  
 3   passengers       float64
 4   distance         float64
 5   air_time         float64
dtypes: float64(3), int64(2), object(1)
memory usage: 107.6+ MB


,dest_airport_id,unique_carrier,month,passengers,distance,air_time
0,10800,0WQ,6,0.0,223.0,91.0
1,12892,0WQ,6,1.0,236.0,45.0
2,14831,0WQ,6,0.0,386.0,63.0
3,10800,0WQ,6,0.0,18.0,9.0
4,10800,0WQ,6,0.0,18.0,9.0




--- 0.22326087951660156 seconds ---


In [78]:
pass_df['unique_carrier'].unique()

array(['0WQ', 'PFQ', '1AQ', '1DQ', '1SQ', '1YQ', 'LF', '7H', '7S', 'ABX',
       '8C', 'CP', 'G7', 'GFQ', 'GL', 'WN', 'WP', 'YR', 'PH', 'PY', '5Y',
       'B6', 'MQ', 'C5', 'F9', 'G4', 'H6', 'AS', 'WRD', '0UQ', '7I', 'AC',
       '9V', 'EK', 'ET', 'EY', 'JL', 'R3', '3S', 'SU', 'VSA', '13Q', '6R',
       'BY', 'A0', 'IG', 'SE', 'SN', 'SVW', 'VJT', '0JQ', '1TQ', '9S',
       'KLQ', 'PO', 'S5', '8V', '8D', 'PPQ', 'HB', 'J5', 'KO', 'Q5',
       'SNK', 'Z3Q', 'JQ', 'AD', 'AI', 'DE', 'NQ', 'OZ', 'RV', 'S4', 'VA',
       'W3', '0Q', '0QQ', '10Q', '17Q', '2AQ', 'B0', 'FI', 'FJ', 'KL',
       'QK', 'RJ', 'TS', 'WW', 'ZX', '12Q', '8R', 'EI', 'PP', 'QT', 'EQ',
       'LX', '4C', '4M', 'AB', 'AF', 'CC', 'CZ', 'LP', 'M3', 'MS', 'NZ',
       'SK', 'TN', 'VB', 'XL', 'Y4', '4W', 'SA', '3U', 'BA', 'DHQ', 'HY',
       'L7', 'LA', 'PK', 'RU', 'TG', 'PEQ', 'UN', 'CM', 'GBQ', 'IB', 'OS',
       'LR', 'CK', 'MT', 'NH', 'AV', 'PBQ', '14Q', 'MU', 'AM', 'C8',
       'VIQ', 'I4', '2E', 'Z3', 'KS', 'AA', 'XP', '

In [79]:
# start time to measure the time of the program execution
start_time = time.time()

# create a new variable with dest_airport_id, unique_carrier and month combined
item_id = []

for num in range(len(pass_df)):
    item = str(pass_df.iloc[num, 0]) + pass_df.iloc[num, 1] + str(pass_df.iloc[num, 2])
    item_id.append(item)

pass_df.insert(loc=2, column='item_id', value=item_id)

# check
pass_df.info()
pass_df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350497 entries, 0 to 2350496
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   dest_airport_id  int64  
 1   unique_carrier   object 
 2   item_id          object 
 3   month            int64  
 4   passengers       float64
 5   distance         float64
 6   air_time         float64
dtypes: float64(3), int64(2), object(2)
memory usage: 125.5+ MB


,dest_airport_id,unique_carrier,item_id,month,passengers,distance,air_time
0,10800,0WQ,108000WQ6,6,0.0,223.0,91.0
1,12892,0WQ,128920WQ6,6,1.0,236.0,45.0
2,14831,0WQ,148310WQ6,6,0.0,386.0,63.0
3,10800,0WQ,108000WQ6,6,0.0,18.0,9.0
4,10800,0WQ,108000WQ6,6,0.0,18.0,9.0




--- 351.2114944458008 seconds ---


In [80]:
pass_df['item_id'].nunique()
pass_df['dest_airport_id'].nunique()

139480

2564

In [81]:
# start time to measure the time of the program execution
start_time = time.time()

# create summary table
sum_pass_df = pd.pivot_table(pass_df, values=['passengers', 'distance', 'air_time'], 
                             index=['item_id', 'dest_airport_id', 'unique_carrier', 'month'],
                                aggfunc=np.sum)

sum_pass_df = sum_pass_df.reset_index()

# check
sum_pass_df.info()
sum_pass_df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139480 entries, 0 to 139479
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   item_id          139480 non-null  object 
 1   dest_airport_id  139480 non-null  int64  
 2   unique_carrier   139480 non-null  object 
 3   month            139480 non-null  int64  
 4   air_time         139480 non-null  float64
 5   distance         139480 non-null  float64
 6   passengers       139480 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 7.4+ MB


,item_id,dest_airport_id,unique_carrier,month,air_time,distance,passengers
0,100055V10,10005,5V,10,954.0,1325.0,16.0
1,100055V2,10005,5V,2,756.0,1146.0,25.0
2,100055V3,10005,5V,3,2531.0,2622.0,124.0
3,100055V4,10005,5V,4,2397.0,2682.0,53.0
4,100055V5,10005,5V,5,3418.0,2725.0,156.0




--- 2.845944881439209 seconds ---


In [82]:
sum_pass_df['air_time'].min()

0.0

In [83]:
# start time to measure the time of the program execution
start_time = time.time()

# create a new variable - speed = distance / air_time (in miles/minutes)
speed = []

for num in range(len(sum_pass_df)):
    if int(sum_pass_df.iloc[num, 4]) == 0:
        speed.append(0)
    else:
        speed.append(sum_pass_df.iloc[num, 5] / sum_pass_df.iloc[num, 4])

sum_pass_df['speed'] = speed
        
# create a new variable with unique_carrier and month combined
carrier_month_id = []

for num in range(len(sum_pass_df)):
    value = sum_pass_df.iloc[num, 2] + str(sum_pass_df.iloc[num, 3])
    carrier_month_id.append(value)

sum_pass_df.insert(loc=1, column='carrier_month_id', value=carrier_month_id)

# check
sum_pass_df.info()
sum_pass_df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139480 entries, 0 to 139479
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   item_id           139480 non-null  object 
 1   carrier_month_id  139480 non-null  object 
 2   dest_airport_id   139480 non-null  int64  
 3   unique_carrier    139480 non-null  object 
 4   month             139480 non-null  int64  
 5   air_time          139480 non-null  float64
 6   distance          139480 non-null  float64
 7   passengers        139480 non-null  float64
 8   speed             139480 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 9.6+ MB


,item_id,carrier_month_id,dest_airport_id,unique_carrier,month,air_time,distance,passengers,speed
0,100055V10,5V10,10005,5V,10,954.0,1325.0,16.0,1.388889
1,100055V2,5V2,10005,5V,2,756.0,1146.0,25.0,1.515873
2,100055V3,5V3,10005,5V,3,2531.0,2622.0,124.0,1.035954
3,100055V4,5V4,10005,5V,4,2397.0,2682.0,53.0,1.118899
4,100055V5,5V5,10005,5V,5,3418.0,2725.0,156.0,0.797250




--- 29.439395427703857 seconds ---


In [84]:
sum_pass_df[['air_time', 'distance', 'passengers', 'speed']].describe()

,air_time,distance,passengers,speed
count,1.394800e+05,1.394800e+05,1.394800e+05,139480.000000
mean,4.044920e+04,1.756827e+04,3.534135e+04,2.013683
std,2.908624e+05,8.062687e+04,2.673444e+05,2.831589
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,5.500000e+01,9.700000e+02,2.000000e+00,0.090725
50%,6.930000e+02,3.112000e+03,2.470000e+02,0.433918
75%,1.149150e+04,1.038600e+04,9.627250e+03,3.468924
max,1.458976e+07,3.446846e+06,1.728704e+07,163.875000


In [85]:
# start time to measure the time of the program execution
start_time = time.time()

# create summary table
carrier_pass = pd.pivot_table(pass_df, values='passengers', index=['unique_carrier', 'month'],
                                aggfunc={'month': [np.sum, np.mean]})

carrier_pass = carrier_pass.reset_index()

# create a new variable with unique_carrier and month combined
carrier_month_id2 = []

for num in range(len(carrier_pass)):
    value2 = carrier_pass.iloc[num, 0] + str(carrier_pass.iloc[num, 1])
    carrier_month_id2.append(value2)

carrier_pass.insert(loc=0, column='carrier_month_id', value=carrier_month_id2)

# remove 'unique_carrier' and 'month' columns
carrier_pass = carrier_pass.drop(columns=['unique_carrier', 'month'])

# check
carrier_pass

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,carrier_month_id,mean,sum
0,02Q1,1.0,2
1,02Q2,2.0,10
2,02Q3,3.0,15
3,02Q4,4.0,28
4,02Q5,5.0,65
...,...,...,...
4219,ZX8,8.0,1656
4220,ZX9,9.0,1791
4221,ZX10,10.0,1920
4222,ZX11,11.0,1771




--- 1.4282755851745605 seconds ---


In [86]:
# start time to measure the time of the program execution
start_time = time.time()

# inner join sum_pass_df with carrier_pass
sum_pass_df = sum_pass_df.merge(carrier_pass, how='inner', on='carrier_month_id')
sum_pass_df = sum_pass_df.rename({'mean': 'mean_passengers', 'sum': 'sum_passengers'}, axis=1)

# check
sum_pass_df.info()
sum_pass_df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139480 entries, 0 to 139479
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   item_id           139480 non-null  object 
 1   carrier_month_id  139480 non-null  object 
 2   dest_airport_id   139480 non-null  int64  
 3   unique_carrier    139480 non-null  object 
 4   month             139480 non-null  int64  
 5   air_time          139480 non-null  float64
 6   distance          139480 non-null  float64
 7   passengers        139480 non-null  float64
 8   speed             139480 non-null  float64
 9   mean_passengers   139480 non-null  float64
 10  sum_passengers    139480 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 12.8+ MB


,item_id,carrier_month_id,dest_airport_id,unique_carrier,month,air_time,distance,passengers,speed,mean_passengers,sum_passengers
0,100055V10,5V10,10005,5V,10,954.0,1325.0,16.0,1.388889,10.0,8580
1,100145V10,5V10,10014,5V,10,82.0,70.0,0.0,0.853659,10.0,8580
2,100645V10,5V10,10064,5V,10,30.0,224.0,0.0,7.466667,10.0,8580
3,101395V10,5V10,10139,5V,10,91.0,323.0,0.0,3.549451,10.0,8580
4,101845V10,5V10,10184,5V,10,274.0,546.0,17.0,1.992701,10.0,8580




--- 0.22041630744934082 seconds ---


In [87]:
# export to csv
sum_pass_df.to_csv('passengers_summary.csv', index=False)

--------------------------------------

#### __Fuel: Feature Engineering__

In [88]:
# import fuel_consumption
fuel = pd.read_csv('fuel_consumption.csv')
fuel.info()
fuel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              3035 non-null   int64  
 1   airline_id         3034 non-null   float64
 2   unique_carrier     3034 non-null   object 
 3   carrier            3035 non-null   object 
 4   carrier_name       3034 non-null   object 
 5   carrier_group_new  3035 non-null   int64  
 6   sdomt_gallons      3034 non-null   float64
 7   satl_gallons       3034 non-null   float64
 8   spac_gallons       3034 non-null   float64
 9   slat_gallons       3033 non-null   float64
 10  sint_gallons       3034 non-null   float64
 11  ts_gallons         3034 non-null   float64
 12  tdomt_gallons      3032 non-null   float64
 13  tint_gallons       3034 non-null   float64
 14  total_gallons      3029 non-null   float64
 15  sdomt_cost         3035 non-null   float64
 16  satl_cost          3034 

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,NaN,0JQ,NaN,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5100.0,0.0,5100.0,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,396216.0,140239.0,536455.0,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848.0,0.0,0.0,0.0,...,522405.0,0.0,0.0,0.0,0.0,522405.0,569497.0,0.0,569497.0,2016


In [89]:
fuel['carrier'].nunique()

65

In [90]:
fuel['unique_carrier'].nunique()

62

In [91]:
fuel.columns

Index(['month', 'airline_id', 'unique_carrier', 'carrier', 'carrier_name',
       'carrier_group_new', 'sdomt_gallons', 'satl_gallons', 'spac_gallons',
       'slat_gallons', 'sint_gallons', 'ts_gallons', 'tdomt_gallons',
       'tint_gallons', 'total_gallons', 'sdomt_cost', 'satl_cost', 'spac_cost',
       'slat_cost', 'sint_cost', 'ts_cost', 'tdomt_cost', 'tint_cost',
       'total_cost', 'year'],
      dtype='object')

In [92]:
# start time to measure the time of the program execution
start_time = time.time()

# remove insignificant columns
fuel_con = fuel.drop(columns=['airline_id','unique_carrier', 'carrier_name',
       'carrier_group_new', 'sdomt_gallons', 'satl_gallons', 'spac_gallons',
       'slat_gallons', 'sint_gallons', 'ts_gallons', 'tdomt_gallons',
       'tint_gallons','sdomt_cost', 'satl_cost', 'spac_cost',
       'slat_cost', 'sint_cost', 'ts_cost', 'tdomt_cost', 'tint_cost','year'
       ])

# check for null values
fuel_con.isnull().sum()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

month            0
carrier          0
total_gallons    6
total_cost       0
dtype: int64



--- 0.012604475021362305 seconds ---


In [93]:
# start time to measure the time of the program execution
start_time = time.time()

# fill the missing values for 'total_gallons' woth median values
fuel_con['total_gallons'] =  fuel_con['total_gallons'].fillna(fuel_con['total_gallons'].median())

# create a new variable with unique_carrier and month combined
carrier_month_id3 = []

for num in range(len(fuel_con)):
    value3 = fuel_con.iloc[num, 1] + str(fuel_con.iloc[num, 0])
    carrier_month_id3.append(value3)

fuel_con.insert(loc=0, column='carrier_month_id', value=carrier_month_id3)

# check
fuel_con.info()
fuel_con.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   carrier_month_id  3035 non-null   object 
 1   month             3035 non-null   int64  
 2   carrier           3035 non-null   object 
 3   total_gallons     3035 non-null   float64
 4   total_cost        3035 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 118.7+ KB


,carrier_month_id,month,carrier,total_gallons,total_cost
0,0JQ1,1,0JQ,3000.0,5100.0
1,0WQ1,1,0WQ,210112.0,536455.0
2,23Q1,1,23Q,0.0,0.0
3,27Q1,1,27Q,0.0,0.0
4,5V1,1,5V,284362.0,569497.0




--- 0.26553940773010254 seconds ---


In [94]:
fuel_con['carrier_month_id'].nunique()

756

In [95]:
# start time to measure the time of the program execution
start_time = time.time()

# create summary table
fuel_summary = pd.pivot_table(fuel_con, values=['total_gallons', 'total_cost'],
                              index=['carrier_month_id','carrier', 'month'],
                                aggfunc={'total_gallons': [np.sum, np.mean],
                                        'total_cost': [np.sum, np.mean]})

fuel_summary = fuel_summary.reset_index()

# check
fuel_summary.info()
fuel_summary.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   (carrier_month_id, )   756 non-null    object 
 1   (carrier, )            756 non-null    object 
 2   (month, )              756 non-null    int64  
 3   (total_cost, mean)     756 non-null    float64
 4   (total_cost, sum)      756 non-null    float64
 5   (total_gallons, mean)  756 non-null    float64
 6   (total_gallons, sum)   756 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 41.5+ KB


carrier_month_id carrier month    total_cost            total_gallons  \
                                          mean        sum          mean   
0             09Q1     09Q     1  2.878729e+06  8636187.0  1.018414e+06   
1            09Q10     09Q    10  2.892003e+06  8676010.0  1.210001e+06   
2            09Q11     09Q    11  2.627896e+06  7883689.0  1.006282e+06   
3            09Q12     09Q    12  2.743162e+06  8229487.0  9.321290e+05   
4             09Q2     09Q     2  2.266957e+06  9067827.0  8.760805e+05   

              
         sum  
0  3055241.0  
1  3630002.0  
2  3018846.0  
3  2796387.0  
4  3504322.0



--- 0.06349611282348633 seconds ---


In [96]:
fuel_summary.columns = ['carrier_month_id', 'carrier', 'month', 'total_cost:mean',
                        'total_cost:sum', 'total_gallons:mean', 'total_gallons:sum']

fuel_summary.info()
fuel_summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   carrier_month_id    756 non-null    object 
 1   carrier             756 non-null    object 
 2   month               756 non-null    int64  
 3   total_cost:mean     756 non-null    float64
 4   total_cost:sum      756 non-null    float64
 5   total_gallons:mean  756 non-null    float64
 6   total_gallons:sum   756 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 41.5+ KB


,carrier_month_id,carrier,month,total_cost:mean,total_cost:sum,total_gallons:mean,total_gallons:sum
0,09Q1,09Q,1,2.878729e+06,8636187.0,1.018414e+06,3055241.0
1,09Q10,09Q,10,2.892003e+06,8676010.0,1.210001e+06,3630002.0
2,09Q11,09Q,11,2.627896e+06,7883689.0,1.006282e+06,3018846.0
3,09Q12,09Q,12,2.743162e+06,8229487.0,9.321290e+05,2796387.0
4,09Q2,09Q,2,2.266957e+06,9067827.0,8.760805e+05,3504322.0


In [97]:
fuel_summary[['total_cost:mean', 'total_cost:sum', 'total_gallons:mean', 'total_gallons:sum']].describe()

,total_cost:mean,total_cost:sum,total_gallons:mean,total_gallons:sum
count,7.560000e+02,7.560000e+02,7.560000e+02,7.560000e+02
mean,4.533527e+07,2.090502e+08,2.476576e+07,1.140044e+08
std,1.192075e+08,5.690868e+08,6.526777e+07,3.112040e+08
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.371850e+05,7.752318e+05,1.206230e+05,3.108282e+05
50%,1.190035e+06,5.020328e+06,5.652571e+05,2.266332e+06
75%,1.670567e+07,7.545949e+07,7.067078e+06,3.533539e+07
max,7.342249e+08,3.671125e+09,3.385212e+08,1.692606e+09


In [73]:
# export to csv
fuel_summary.to_csv('fuel_summary.csv', index=False)